In [3]:
import pandas as pd
import pyranges as pr
import pickle
import os
import sys

In [21]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/hematokytos/reference/token_map.csv"

df = pd.read_csv(fpath)
print(f"{df.shape=}")
df.head()



df.shape=(25426, 7)


,gene_id,token_id,gene_name,gene_biotype,Chromosome,Start,End
0,<pad>,0,NaN,NaN,NaN,NaN,NaN
1,<mask>,1,NaN,NaN,NaN,NaN,NaN
2,ENSG00000000003,2,TSPAN6,protein_coding,X,100627107.0,100639991.0
3,ENSG00000000005,3,TNMD,protein_coding,X,100584935.0,100599885.0
4,ENSG00000000419,4,DPM1,protein_coding,20,50934866.0,50959140.0


In [25]:
duplicate_rows = df[df['gene_name'].duplicated(keep=False)]
duplicate_rows = duplicate_rows[duplicate_rows['gene_name'].notna()]

duplicate_rows.sort_values(by='gene_name')

,gene_id,token_id,gene_name,gene_biotype,Chromosome,Start,End
25384,ENSG00000287542,25384,HERC3,protein_coding,4,88523809.0,88708450.0
7660,ENSG00000138641,7660,HERC3,protein_coding,4,88592433.0,88708541.0
24647,ENSG00000280987,24647,MATR3,protein_coding,5,139273751.0,139331671.0
362,ENSG00000015479,362,MATR3,protein_coding,5,139293673.0,139331677.0
25307,ENSG00000285723,25307,MKKS,protein_coding,20,10420545.0,10420737.0
5714,ENSG00000125863,5714,MKKS,protein_coding,20,10401008.0,10434222.0
25285,ENSG00000285508,25285,MKKS,protein_coding,20,10413519.0,10431922.0
22203,ENSG00000264809,22203,Metazoa_SRP,misc_RNA,1,178484407.0,178484682.0
17421,ENSG00000203286,17421,Metazoa_SRP,misc_RNA,9,128745856.0,128746138.0
18673,ENSG00000214305,18673,Metazoa_SRP,misc_RNA,3,39161269.0,39161520.0


In [13]:
def load_gtf(gtf_path):
    """
    Loads gene information from a GTF file into a Pandas DataFrame.

    Args:
        gtf_path: The path to the GTF file.

    Returns:
        A Pandas DataFrame containing the following columns for each gene:
            - gene_name
            - gene_id
            - gene_biotype
            - Chromosome
            - Start
            - End
    """

    keep_columns = ['gene_name', 'gene_id', 'gene_biotype', 'Chromosome', 'Start', 'End']
    gdf = pr.read_gtf(gtf_path).as_df()
    gdf = gdf[(gdf['Feature'] == 'gene') & gdf['gene_id'].notna()][keep_columns]
    return gdf.drop_duplicates().reset_index(drop=True)

    
gtf_path = "/scratch/indikar_root/indikar1/cstansbu/hematokytos/reference/annotations.gtf"
gdf = load_gtf(gtf_path)
print(f"{gdf.shape=}")
gdf.head()

/home/cstansbu/miniconda3/envs/bioinf/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


gdf.shape=(61860, 6)


,gene_name,gene_id,gene_biotype,Chromosome,Start,End
0,ATAD3B,ENSG00000160072,protein_coding,1,1471764,1497848
1,NaN,ENSG00000234396,lncRNA,1,2212522,2220738
2,MTND1P23,ENSG00000225972,unprocessed_pseudogene,1,629061,629433
3,MTCO3P12,ENSG00000198744,unprocessed_pseudogene,1,634375,634922
4,DDX11L17,ENSG00000279928,unprocessed_pseudogene,1,182695,184174


In [18]:
def load_gene_tokenization(token_dictionary_file):
    """
    Loads gene tokenization data from a pickle file into a DataFrame.

    Args:
        token_dictionary_file (str): Path to the pickle file containing the gene-token dictionary.

    Returns:
        pd.DataFrame: A DataFrame with 'Ensembl ID' and 'Token' columns.
    """

    with open(token_dictionary_file, "rb") as f:
        gene_token_dict = pickle.load(f)

    return pd.DataFrame(list(gene_token_dict.items()), columns=['gene_id', 'token_id'])


token_dictionary_file = '/scratch/indikar_root/indikar1/cstansbu/hematokytos/reference/token_dictionary.pkl'
tokens = load_gene_tokenization(token_dictionary_file)
print(tokens.shape)
tokens

(25426, 2)


,gene_id,token_id
0,<pad>,0
1,<mask>,1
2,ENSG00000000003,2
3,ENSG00000000005,3
4,ENSG00000000419,4
...,...,...
25421,ENSGR0000197976,25421
25422,ENSGR0000198223,25422
25423,ENSGR0000205755,25423
25424,ENSGR0000214717,25424


In [20]:
df = pd.merge(
    tokens,
    gdf,
    how='left',

)

print(df.shape)
df.head()



(25426, 7)


,gene_id,token_id,gene_name,gene_biotype,Chromosome,Start,End
0,<pad>,0,NaN,NaN,NaN,NaN,NaN
1,<mask>,1,NaN,NaN,NaN,NaN,NaN
2,ENSG00000000003,2,TSPAN6,protein_coding,X,100627107.0,100639991.0
3,ENSG00000000005,3,TNMD,protein_coding,X,100584935.0,100599885.0
4,ENSG00000000419,4,DPM1,protein_coding,20,50934866.0,50959140.0
